In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.linear_models import LinearRegression
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.optim as optim
import torch.nn.functional import F
import torchvision
from torchvision import transformers
import time
import matplotlib.pyplot as plt

In [ ]:
data = [1.0,2.0,3.0]
tensor = torch.Tensor(data)
print("1D Data: ")
print(tensor)

data = [[1., 2., 3.], [4.,5., 6.]]
tensor = torch.Tensor(data)
print("2D Data: ")
print(tensor)

In [ ]:
rand = torch.randn(2,3,3)
rand

In [ ]:
X = np.arange(0, 10, 0.5)
epsilon = np.random.normal(0, 0.5, len(X))
y = (2 + 3*X)

plt.scatter(X, y, s=20)
plt.title("Randomly Generated Data")
plt.show()

In [ ]:
X, y = X.reshape(-1,1), y.reshape(-1,1)

model = LinearRegression()
model.fit(X, y)
print("W_1:", model.coef_[0][0])
print("W_0:", model.intercept_[0])